In [72]:
import os, sys, pytz
import pandas as pd
from django.db import DEFAULT_DB_ALIAS, connections
from django.db.migrations.loader import MigrationLoader
from django.db.migrations.recorder import MigrationRecorder

sys.path.append('../') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

tz = pytz.timezone('Africa/Gaborone')
connection = connections[DEFAULT_DB_ALIAS]
loader = MigrationLoader(connection)

In [73]:
migrations = MigrationRecorder.Migration.objects.filter(app__startswith='flourish')

In [74]:
records = []
for migration in migrations:
    try:
        operations = loader.get_migration_by_prefix(migration.app, migration.name).operations
    except KeyError:
        continue
    else:
        for operation in operations:
            operation_type, _, details = operation.deconstruct()
            dt_applied = migration.applied.astimezone(tz)
            record = {'date_applied': dt_applied.strftime('%d-%m-%Y %H:%M %p'),
                      'operation_applied': operation.describe(),}
            if operation_type == 'CreateModel':
                model_name = details.get('name')
                fields = details.get('fields')     
                field_list = [field[0] for field in fields]
                record.update(
                    {'form_name': model_name,
                     'field(s)': ', '.join(field_list)})
            elif operation_type == 'RenameModel':
                record.update({
                    'form_name': f'old_name: {details.get("old_name")} -> new_name: {details.get("new_name")}',
                    'field(s)': ''
                })
            elif operation_type in ['RemoveField', 'AddField', 'AlterField']:
                model_name = details.get('model_name')
                record.update({
                    'form_name': model_name,
                    'field(s)': details.get('name')
                })
            elif operation_type == 'DeleteModel':
                model_name = details.get('name')
                record.update({
                    'form_name': model_name,
                    'field(s)': ''
                })
            elif operation_type == 'RenameField':
                model_name = details.get('model_name')
                record.update({
                    'form_name': model_name,
                    'field(s)': f'old_name: {details.get("old_name")} -> new_name: {details.get("new_name")}'
                })
            elif operation_type in ['AlterModelManagers', 'AlterModelOptions', 'RunPython', 'AlterUniqueTogether']:
                continue
            else:
                print(operation_type, details, migration.applied, operation.describe(), operation.deconstruct(), )
                print('\n')
            records.append(record)

In [75]:
df = pd.DataFrame(records)
df.to_csv('migrations_details.csv', encoding='utf-8', index=False)